In [1]:
import os 
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
from tensorflow.keras.datasets import mnist

physical_devices = tf.config.list_physical_devices('GPU')
print('Physical Devices', physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], True)

Physical Devices [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


### Load MNIST data

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [3]:
x_train = x_train.reshape(-1, 28 * 28).astype('float32') /  255.
x_test = x_test.reshape(-1, 28 * 28).astype('float32') /  255.

In [4]:
x_train.shape

(60000, 784)

### Recreating Dense layer of Keras

There are three functions here to work. The ____init____ method, the __build__ method and the __call__ method. <br>

If you have used Pytorch before, the __call__ method is the same as the forward method in Pytorch.
<br>
The __build__ method is where you define all your variables. You can define the variables in the __init__ method, but if we do not know the shape of the input beforehand, then it is recommended to do it in the __build__ method. So, we can always stick to defining the weights in the __build__ method, rather than in the __init__ method. 


##### The __add_weight__ method

This adds a new variable to our layer and we can set whether the layer is trainable or not. 
Link to documentation: https://www.tensorflow.org/api_docs/python/tf/keras/layers/Layer#add_weight


In [5]:
class Dense(layers.Layer):
    def __init__(self, units):
        super(Dense, self).__init__()
        self.units = units
    
    def build(self, input_shape):
        self.w = self.add_weight(
                name='w',
                shape=(input_shape[-1], self.units), #Units -> Number of ouput units,
                initializer='random_normal',
                trainable=True
            )

        self.b = self.add_weight(
            name='b', shape = (self.units, ), initializer='zeros', trainable=True
        )
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b   

### Recreate the ReLU layer

He we won't require any variables to be declared because the ReLU is a parameter less layer. It only requires to trim away the non-zero values and clamp them to 0 or any other linear function. 

In [6]:
class MyReLU(layers.Layer):
    def __init__(self):
        super(MyReLU, self).__init__()
    def call(self, x):
        return tf.math.maximum(x, 0)

### Subclassing keras.Model to create custom Models

We can build multiple

In [7]:
# class MyModel(keras.Model):
#     def __init__(self, num_classes = 10):
#         super(MyModel, self).__init__()
#         self.dense1 = layers.Dense(64)
#         self.dense2 = layers.Dense(num_classes)
        
#     def call(self, input_tensor):
#         x = tf.nn.relu(self.dense1(input_tensor))
#         return self.dense2(x)

class MyModel(keras.Model):
    def __init__(self, num_classes = 10):
        super(MyModel, self).__init__()
        self.dense1 = Dense(64)
        self.dense2 = Dense(num_classes)
        self.relu = MyReLU()
        
    def call(self, input_tensor):
        x = self.relu(self.dense1(input_tensor))
        return self.dense2(x)


In [8]:
model = MyModel()

In [9]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(),
    metrics = ['accuracy']
)

In [10]:
model.fit(x_train, y_train, batch_size=32, epochs=2, verbose=2)

Epoch 1/2
1875/1875 - 4s - loss: 0.3462 - accuracy: 0.9046 - 4s/epoch - 2ms/step
Epoch 2/2
1875/1875 - 3s - loss: 0.1640 - accuracy: 0.9525 - 3s/epoch - 2ms/step


In [11]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.1335 - accuracy: 0.9591


[0.13350139558315277, 0.9591000080108643]

In [12]:
model.save_weights('saved_model/')

In [13]:
model = MyModel()

In [14]:
model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer = keras.optimizers.Adam(),
    metrics = ['accuracy']
)

In [15]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 2.3151 - accuracy: 0.0590


[2.3151166439056396, 0.05900000035762787]

In [16]:
model.load_weights('saved_model/')

In [17]:
model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.1335 - accuracy: 0.9591


[0.13350139558315277, 0.9591000080108643]

In [19]:
model.save('complete_saved_model/')

INFO:tensorflow:Assets written to: complete_saved_model/assets


In [20]:
model = keras.models.load_model('complete_saved_model/')

In [21]:
model